In [71]:
import pandas as pd

In [72]:
df = pd.read_csv("/content/train.csv")

In [73]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [74]:
df.shape

(20800, 5)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [76]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [77]:
df = df.dropna()

In [78]:
df.shape

(18285, 5)

In [79]:
X = df.drop("label" , axis = 1)

In [80]:
y = df["label"]

In [81]:
import tensorflow as tf

In [82]:
tf.__version__

'2.17.0'

In [83]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [84]:
voc_size = 5000

In [85]:
messages = X.copy()

In [86]:
messages.reset_index(drop=True, inplace=True)

In [87]:
print(messages.index)

RangeIndex(start=0, stop=18285, step=1)


In [88]:
import nltk
import re
from nltk.corpus import stopwords

In [89]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0 , len(messages)):
  review = re.sub('[^a-zA-Z]' , ' ' , messages['title'][i] )
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [91]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [ ]:
one_hot_repr = [one_hot(words , voc_size) for words in corpus]
one_hot_repr

# Embedding Representation

In [102]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr , padding = 'pre' , maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ...    2 2566 4038]
 [   0    0    0 ... 2773 3966 4498]
 [   0    0    0 ... 1760 3490 4440]
 ...
 [   0    0    0 ... 3210  542 1949]
 [   0    0    0 ... 2895 1966 4536]
 [   0    0    0 ... 4602 3509 4337]]


In [103]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3942,
       1265, 4565, 3517, 4664, 3861, 2786,    2, 2566, 4038], dtype=int32)

In [104]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4896, 2076, 4310, 3168, 2773, 3966, 4498], dtype=int32)

In [109]:
# creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_shape=(sent_length,)))
model.add(LSTM(100))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [110]:
len(embedded_docs), y.shape

(18285, (18285,))

In [111]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [112]:
X_final.shape , y_final.shape

((18285, 20), (18285,))

In [113]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final , y_final , test_size = 0.33 , random_state = 42)

In [114]:
# model training
model.fit(X_train , y_train , validation_data = (X_test , y_test) , epochs = 10 , batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 62ms/step - accuracy: 0.7885 - loss: 0.4281 - val_accuracy: 0.9087 - val_loss: 0.2033
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9410 - loss: 0.1423 - val_accuracy: 0.9152 - val_loss: 0.2014
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.9638 - loss: 0.1008 - val_accuracy: 0.9167 - val_loss: 0.2200
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.9792 - loss: 0.0608 - val_accuracy: 0.9097 - val_loss: 0.2336
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9845 - loss: 0.0476 - val_accuracy: 0.9140 - val_loss: 0.2488
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - accuracy: 0.9933 - loss: 0.0276 - val_accuracy: 0.9120 - val_loss: 0.3156
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9965 - loss: 0.0145 - val_accuracy: 0.9074 - val_loss: 0.3699
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.9974 - loss: 0.0110 - v

In [115]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [116]:
y_pred = np.where(y_pred > 0.5 , 1 ,0)

In [117]:
from sklearn.metrics import confusion_matrix

In [119]:
confusion_matrix(y_test , y_pred)

array([[3087,  332],
       [ 205, 2411]])

In [120]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.9110190555095278